In [1]:
# !wget https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/speaker_recognition/conf/SpeakerNet_recognition_3x2x512.yaml

In [2]:
# !cat conf/SpeakerNet_recognition_3x2x512.yaml

In [3]:
# !pip3 install nemo-toolkit[asr]==1.0.0b1

In [4]:
# !pip3 install torchaudio>=0.6.0 -f https://download.pytorch.org/whl/torch_stable.html

In [5]:
# !git pull

In [6]:
import nemo
import nemo.collections.asr as nemo_asr
from omegaconf import OmegaConf

[NeMo W 2020-10-25 01:55:34 nemo_logging:349] /home/ubuntu/.local/lib/python3.6/site-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
      warnings.warn(msg)
    
[NeMo W 2020-10-25 01:55:34 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToCharDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-25 01:55:34 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioToBPEDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-25 01:55:34 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text.AudioLabelDataset'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2020-10-25 01:55:34 experimental:28] Module <class 'nemo.collections.asr.data.audio_to_text._TarredAudioToTextDataset'

In [7]:
config = OmegaConf.load('SpeakerNet_recognition_3x2x512.yaml')
print(OmegaConf.to_yaml(config))

name: SpeakerNet
sample_rate: 16000
repeat: 2
dropout: 0.5
separable: true
n_filters: 512
model:
  train_ds:
    manifest_filepath: ???
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
  validation_ds:
    manifest_filepath: ???
    sample_rate: 16000
    labels: null
    batch_size: 128
    shuffle: false
    time_length: 8
  test_ds:
    manifest_filepath: ???
    sample_rate: 16000
    labels: null
    batch_size: 1
    shuffle: false
    time_length: 8
    embedding_dir: .
  preprocessor:
    _target_: nemo.collections.asr.modules.AudioToMelSpectrogramPreprocessor
    normalize: per_feature
    window_size: 0.02
    sample_rate: 16000
    window_stride: 0.01
    window: hann
    features: 64
    n_fft: 512
    frame_splicing: 1
    dither: 1.0e-05
    stft_conv: false
  encoder:
    _target_: nemo.collections.asr.modules.ConvASREncoder
    feat_in: 64
    activation: relu
    conv_mask: true
    jasper:
    - filters: 512
      repeat:

In [8]:
# !wget https://api.ngc.nvidia.com/v2/models/nvidia/nemospeechmodels/versions/1.0.0a5/files/SpeakerNet_verification.nemo

In [9]:
verification_model = nemo_asr.models.ExtractSpeakerEmbeddingsModel.restore_from('SpeakerNet_verification.nemo')

[NeMo W 2020-10-25 01:55:35 modelPT:102] Please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /data/samsungSSD/NVIDIA/repos/NeMo/examples/speaker_recognition/myExps/datasets/combined/train_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
    
[NeMo W 2020-10-25 01:55:35 modelPT:109] Please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /data/samsungSSD/NVIDIA/repos/NeMo/examples/speaker_recognition/myExps/datasets/voxceleb/train/small_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 128
    shuffle: false
    time_length: 8
    
[NeMo W 2020-10-25 01:55:35 nemo_logging:349] /home/ubuntu/.local/lib/python3.6/site-packages/hy

[NeMo I 2020-10-25 01:55:35 features:241] PADDING: 16
[NeMo I 2020-10-25 01:55:35 features:258] STFT using torch
[NeMo I 2020-10-25 01:55:35 label_models:86] Training with Softmax-CrossEntropy loss
[NeMo I 2020-10-25 01:55:35 modelPT:237] Model ExtractSpeakerEmbeddingsModel was successfully restored from SpeakerNet_verification.nemo.


In [10]:
import malaya_speech
import malaya_speech.train.model.speakernet as speakernet
import tensorflow as tf


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [11]:
inputs = tf.placeholder(tf.float32, [None, None, 64])
inputs_length = tf.placeholder(tf.int32, [None])

In [12]:
model = speakernet.Model(inputs, inputs_length, mode = 'eval')
model.logits


Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use `tf.keras.layers.SeparableConv1D` instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.Dense instead.


<tf.Tensor 'dense_2/MatMul:0' shape=(?, 7205) dtype=float32>

In [13]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [14]:
variables = [v for v in tf.get_collection('variables')]
encoder = [v for v in variables if 'encoder' in v.name]
decoder = [v for v in variables if 'encoder' not in v.name]
decoder

[<tf.Variable 'dense/kernel:0' shape=(3000, 512) dtype=float32_ref>,
 <tf.Variable 'dense/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/gamma:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/beta:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/moving_mean:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization/moving_variance:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'dense_1/kernel:0' shape=(512, 512) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/gamma:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/beta:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/moving_mean:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'batch_normalization_1/moving_variance:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(512, 7205) dtype=float32_ref>]

In [15]:
encoder[0].load(verification_model.encoder.encoder[0].res[0][0].conv.weight.permute([2,1,0]).detach().numpy())
encoder[1].load(verification_model.encoder.encoder[0].res[0][1].weight.detach().numpy())
encoder[2].load(verification_model.encoder.encoder[0].res[0][1].bias.detach().numpy())
encoder[3].load(verification_model.encoder.encoder[0].res[0][1].running_mean.detach().numpy())
encoder[4].load(verification_model.encoder.encoder[0].res[0][1].running_var.detach().numpy())
encoder[5].load(verification_model.encoder.encoder[0].mconv[0].conv.weight.permute([2,0,1]).detach().numpy())
encoder[6].load(verification_model.encoder.encoder[0].mconv[1].conv.weight.permute([2,1,0]).detach().numpy())
encoder[7].load(verification_model.encoder.encoder[0].mconv[2].weight.detach().numpy())
encoder[8].load(verification_model.encoder.encoder[0].mconv[2].bias.detach().numpy())
encoder[9].load(verification_model.encoder.encoder[0].mconv[2].running_mean.detach().numpy())
encoder[10].load(verification_model.encoder.encoder[0].mconv[2].running_var.detach().numpy())

encoder[11].load(verification_model.encoder.encoder[1].mconv[0].conv.weight.permute([2,0,1]).detach().numpy())
encoder[12].load(verification_model.encoder.encoder[1].mconv[1].conv.weight.permute([2,1,0]).detach().numpy())
encoder[13].load(verification_model.encoder.encoder[1].mconv[2].weight.detach().numpy())
encoder[14].load(verification_model.encoder.encoder[1].mconv[2].bias.detach().numpy())
encoder[15].load(verification_model.encoder.encoder[1].mconv[2].running_mean.detach().numpy())
encoder[16].load(verification_model.encoder.encoder[1].mconv[2].running_var.detach().numpy())
encoder[17].load(verification_model.encoder.encoder[1].res[0][0].conv.weight.permute([2,1,0]).detach().numpy())
encoder[18].load(verification_model.encoder.encoder[1].res[0][1].weight.detach().numpy())
encoder[19].load(verification_model.encoder.encoder[1].res[0][1].bias.detach().numpy())
encoder[20].load(verification_model.encoder.encoder[1].res[0][1].running_mean.detach().numpy())
encoder[21].load(verification_model.encoder.encoder[1].res[0][1].running_var.detach().numpy())
encoder[22].load(verification_model.encoder.encoder[1].mconv[5].conv.weight.permute([2,0,1]).detach().numpy())
encoder[23].load(verification_model.encoder.encoder[1].mconv[6].conv.weight.permute([2,1,0]).detach().numpy())
encoder[24].load(verification_model.encoder.encoder[1].mconv[7].weight.detach().numpy())
encoder[25].load(verification_model.encoder.encoder[1].mconv[7].bias.detach().numpy())
encoder[26].load(verification_model.encoder.encoder[1].mconv[7].running_mean.detach().numpy())
encoder[27].load(verification_model.encoder.encoder[1].mconv[7].running_var.detach().numpy())

encoder[28].load(verification_model.encoder.encoder[2].mconv[0].conv.weight.permute([2,0,1]).detach().numpy())
encoder[29].load(verification_model.encoder.encoder[2].mconv[1].conv.weight.permute([2,1,0]).detach().numpy())
encoder[30].load(verification_model.encoder.encoder[2].mconv[2].weight.detach().numpy())
encoder[31].load(verification_model.encoder.encoder[2].mconv[2].bias.detach().numpy())
encoder[32].load(verification_model.encoder.encoder[2].mconv[2].running_mean.detach().numpy())
encoder[33].load(verification_model.encoder.encoder[2].mconv[2].running_var.detach().numpy())
encoder[34].load(verification_model.encoder.encoder[2].res[0][0].conv.weight.permute([2,1,0]).detach().numpy())
encoder[35].load(verification_model.encoder.encoder[2].res[0][1].weight.detach().numpy())
encoder[36].load(verification_model.encoder.encoder[2].res[0][1].bias.detach().numpy())
encoder[37].load(verification_model.encoder.encoder[2].res[0][1].running_mean.detach().numpy())
encoder[38].load(verification_model.encoder.encoder[2].res[0][1].running_var.detach().numpy())
encoder[39].load(verification_model.encoder.encoder[2].mconv[5].conv.weight.permute([2,0,1]).detach().numpy())
encoder[40].load(verification_model.encoder.encoder[2].mconv[6].conv.weight.permute([2,1,0]).detach().numpy())
encoder[41].load(verification_model.encoder.encoder[2].mconv[7].weight.detach().numpy())
encoder[42].load(verification_model.encoder.encoder[2].mconv[7].bias.detach().numpy())
encoder[43].load(verification_model.encoder.encoder[2].mconv[7].running_mean.detach().numpy())
encoder[44].load(verification_model.encoder.encoder[2].mconv[7].running_var.detach().numpy())

encoder[45].load(verification_model.encoder.encoder[3].mconv[0].conv.weight.permute([2,0,1]).detach().numpy())
encoder[46].load(verification_model.encoder.encoder[3].mconv[1].conv.weight.permute([2,1,0]).detach().numpy())
encoder[47].load(verification_model.encoder.encoder[3].mconv[2].weight.detach().numpy())
encoder[48].load(verification_model.encoder.encoder[3].mconv[2].bias.detach().numpy())
encoder[49].load(verification_model.encoder.encoder[3].mconv[2].running_mean.detach().numpy())
encoder[50].load(verification_model.encoder.encoder[3].mconv[2].running_var.detach().numpy())
encoder[51].load(verification_model.encoder.encoder[3].res[0][0].conv.weight.permute([2,1,0]).detach().numpy())
encoder[52].load(verification_model.encoder.encoder[3].res[0][1].weight.detach().numpy())
encoder[53].load(verification_model.encoder.encoder[3].res[0][1].bias.detach().numpy())
encoder[54].load(verification_model.encoder.encoder[3].res[0][1].running_mean.detach().numpy())
encoder[55].load(verification_model.encoder.encoder[3].res[0][1].running_var.detach().numpy())
encoder[56].load(verification_model.encoder.encoder[3].mconv[5].conv.weight.permute([2,0,1]).detach().numpy())
encoder[57].load(verification_model.encoder.encoder[3].mconv[6].conv.weight.permute([2,1,0]).detach().numpy())
encoder[58].load(verification_model.encoder.encoder[3].mconv[7].weight.detach().numpy())
encoder[59].load(verification_model.encoder.encoder[3].mconv[7].bias.detach().numpy())
encoder[60].load(verification_model.encoder.encoder[3].mconv[7].running_mean.detach().numpy())
encoder[61].load(verification_model.encoder.encoder[3].mconv[7].running_var.detach().numpy())

encoder[62].load(verification_model.encoder.encoder[4].mconv[0].conv.weight.permute([2,0,1]).detach().numpy())
encoder[63].load(verification_model.encoder.encoder[4].mconv[1].conv.weight.permute([2,1,0]).detach().numpy())
encoder[64].load(verification_model.encoder.encoder[4].mconv[2].weight.detach().numpy())
encoder[65].load(verification_model.encoder.encoder[4].mconv[2].bias.detach().numpy())
encoder[66].load(verification_model.encoder.encoder[4].mconv[2].running_mean.detach().numpy())
encoder[67].load(verification_model.encoder.encoder[4].mconv[2].running_var.detach().numpy())

Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


In [16]:
decoder[0].load(verification_model.decoder.emb_layers[0][0].weight.T.detach().numpy())
decoder[1].load(verification_model.decoder.emb_layers[0][0].bias.detach().numpy())
decoder[4].load(verification_model.decoder.emb_layers[0][1].running_mean.detach().numpy())
decoder[5].load(verification_model.decoder.emb_layers[0][1].running_var.detach().numpy())

decoder[6].load(verification_model.decoder.emb_layers[1][0].weight.T.detach().numpy())
decoder[7].load(verification_model.decoder.emb_layers[1][0].bias.detach().numpy())
decoder[10].load(verification_model.decoder.emb_layers[1][1].running_mean.detach().numpy())
decoder[11].load(verification_model.decoder.emb_layers[1][1].running_var.detach().numpy())
decoder[12].load(verification_model.decoder.final.weight.detach().numpy().T)

In [17]:
saver = tf.train.Saver()
saver.save(sess, 'speakernet/model.ckpt')

'speakernet/model.ckpt'

In [18]:
import malaya_speech.config

config = malaya_speech.config.speakernet_featurizer_config
featurizer = malaya_speech.featurization.SpeakerNetFeaturizer(config)

In [19]:
from glob import glob

speakers = glob('speech/example-speaker/*.wav')
speakers

['speech/example-speaker/khalil-nooh.wav',
 'speech/example-speaker/husein-zolkepli.wav',
 'speech/example-speaker/mas-aisyah.wav',
 'speech/example-speaker/shafiqah-idayu.wav']

In [20]:
wavs = [malaya_speech.load(f, sr = 16000)[0] for f in speakers]
vectors = [featurizer.vectorize(w) for w in wavs]
padded, l = malaya_speech.padding.sequence_nd(vectors, dim = 0, return_len = True)
padded.shape

(4, 564, 64)

In [21]:
logits = sess.run(model.logits, feed_dict = {inputs: padded, inputs_length: l})
logits.shape

(4, 7205)

In [22]:
from scipy.spatial.distance import cdist

1 - cdist(logits, logits, metric = 'cosine')

array([[1.        , 0.91986295, 0.85895558, 0.85036781],
       [0.91986295, 1.        , 0.85086457, 0.86070392],
       [0.85895558, 0.85086457, 1.        , 0.88895706],
       [0.85036781, 0.86070392, 0.88895706, 1.        ]])

In [23]:
!tar -czvf speakernet.tar.gz speakernet

speakernet/
speakernet/model.ckpt.index
speakernet/model.ckpt.meta
speakernet/model.ckpt.data-00000-of-00001
speakernet/checkpoint
